In [ ]:
import geopandas as gpd
from geopy.geocoders import Nominatim
import pandas as pd

In [ ]:
zip_to_fips = pd.read_csv('zip-to-fips.csv')
fips_center = pd.read_csv('fips-county-center.csv', dtype={'fips_code': str})
fips_center.head()

In [ ]:
wf = gpd.read_file('wf.geojson')
print(len(wf), "Whole Foods Stores")
dg = gpd.read_file('dg.geojson')
print(len(dg), "Dollar General Stores")
dg.head()

In [ ]:
import numpy as np
def get_fips(lon, lat):
    # find point closest to the input coordinates
    distances = np.sqrt((fips_center['lng'] - lon)**2 + (fips_center['lat'] - lat)**2)
    closest = fips_center.iloc[distances.idxmin()]
    return closest['fips_code']


In [ ]:
wf['fips'] = wf.apply(lambda row: get_fips(row['geometry'].x, row['geometry'].y), axis=1)
dg['fips'] = dg.apply(lambda row: get_fips(row['geometry'].x, row['geometry'].y), axis=1)

In [ ]:
import json
with open('president_results.json') as f:
    data = json.load(f)
    #print(data['data'].keys())
state_results = pd.DataFrame(data['data']['races'])


In [ ]:
# create a dictionary with key of 'fips' and 'results'
fips_results = {}
for i in range(len(state_results)):
    counties = state_results['counties'][i]
    for county in counties:
        fips = county['fips']
        results = county['results']
        fips_results[fips] = results


In [ ]:
wf_results = gpd.GeoDataFrame(geometry=wf['geometry'], data=wf['fips'])
wf_results['biden'] = wf_results.apply(lambda row: fips_results.get(row['fips'], {})['bidenj'], axis=1)
wf_results['trump'] = wf_results.apply(lambda row: fips_results.get(row['fips'], {})['trumpd'], axis=1)
wf_results['winner'] = wf_results.apply(lambda row: 'biden' if row['biden'] > row['trump'] else 'trump', axis=1)
wf_results['biden_percent'] = wf_results['biden'] / (wf_results['biden'] + wf_results['trump'])
wf_results['trump_percent'] = wf_results['trump'] / (wf_results['biden'] + wf_results['trump'])
wf_results

dg_results = gpd.GeoDataFrame(geometry=dg['geometry'], data=dg['fips'])
dg_results['biden'] = dg_results.apply(lambda row: fips_results.get(row['fips'], {})['bidenj'], axis=1)
dg_results['trump'] = dg_results.apply(lambda row: fips_results.get(row['fips'], {})['trumpd'], axis=1)
dg_results['winner'] = dg_results.apply(lambda row: 'biden' if row['biden'] > row['trump'] else 'trump', axis=1)
dg_results['biden_percent'] = dg_results['biden'] / (dg_results['biden'] + dg_results['trump'])
dg_results['trump_percent'] = dg_results['trump'] / (dg_results['biden'] + dg_results['trump'])
dg_results

In [ ]:
# write fips_results to a file
with open('election_results_by_fips.json', 'w') as f:
    json.dump(fips_results, f)

In [ ]:
# what fraction of WF stores are in counties that voted for Biden?
print(wf_results['winner'].value_counts(normalize=True))

# what fraction of DG stores are in counties that voted for Biden?
print(dg_results['winner'].value_counts(normalize=True))

In [ ]:
# get results for each whole foods
for fips in wf['fips']:
    try:
        print(fips_results[str(fips)])
    except:
        print("No results for", fips)


In [ ]:
# print all keys and values for the first entry
for k, v in results.iloc[0].items():
    print(k, v)

In [ ]:
import overpy

# Define the query area
area = "[out:json];(node['name'~'Whole Foods'][shop=yes];>;>;);out body;"

# Create the Overpass API object
api = overpy.Overpass()

# Run the query
result = api.query(area)

# Extract the Whole Foods locations
whole_foods_locations = []
for node in result.nodes:
  if node.tags.get("shop") == "yes" and node.tags.get("name").lower().startswith("whole foods"):
    whole_foods_locations.append({
      "name": node.tags.get("name"),
      "latitude": node.lat,
      "longitude": node.lon
    })

# Print the locations
for location in whole_foods_locations:
  print(f"Name: {location['name']}, Latitude: {location['latitude']}, Longitude: {location['longitude']}")


In [ ]:
!pip install overpass


In [ ]:
import overpy


def get_locations(store_name):
    # Define the bounding box coordinates for the mainland US
    bbox = (24, -128, 50, -66.0)

    # Convert the bounding box coordinates to Overpass API format
    bbox_str = ','.join(map(str, bbox))

    # Define the Overpass query
    query = '''
    [out:json];
    node["name"="{store_name}"]({bbox});
    out;
    '''.format(bbox=bbox_str, store_name=store_name)
    print(query)
    # Create Overpass API object
    api = overpy.Overpass()

    # Send the query to Overpass API
    result = api.query(query)

    # Extract the locations
    locations = []
    for node in result.nodes:
        locations.append([float(node.lon), float(node.lat)])
    return locations

In [ ]:
test = get_locations('Whole Foods Market')

In [ ]:
test